In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
page = requests.get("https://sofifa.com/players?offset=0")

In [3]:
page = requests.get("https://sofifa.com/players?offset=0")
soup = bs(page.content, 'lxml')
t=soup.find('table', {'class': 'table table-hover persist-area'}).find('tbody').find_all('a')
name={}
for i in t:
    #print(i['href'],i.text)
    if i['href'][0:8]=='/player/':
        name[i.text]=i['href']


url_main='https://sofifa.com'

In [4]:
main={}
for url in name.items():
    #print(url[0])
    temp=url_main+url[1]
    page = requests.get(temp)
    soup = bs(page.content, 'lxml')
    th=soup.find_all('ul',{'class': 'pl'})
    l=[]
    for j,i in enumerate(th):
        if j!=0 and j!=1 and j!=2 and j!=10:
            l.append(i.text.strip().split())
    
    d={}
    i=0
    t=0
    for u in range(len(l)):
        i=0
        t=0
        while(i<len(l[u])):
            if l[u][i].isdigit():
                j=i+1
                if j==len(l[u])-1:
                    d[' '.join(l[u][i+1:min(len(l[u]),j+1)])]=int(l[u][i])
                    #print(l[u][i])
                    break
                else:
                    while(l[u][j].isdigit()==False):
                            j=j+1
                            if j==len(l[u])-1:
                                d[' '.join(l[u][i+1:min(len(l[u]),j+1)])]=int(l[u][i])
                                t=-1
                                break
                    if t==-1:
                        break
                    d[' '.join(l[u][i+1:min(len(l[u]),j)])]=int(l[u][i])
                    i=j
            else:
                i=i+1
                    #print(l[u][i])
    main[url[0]]=d  

In [63]:
k=['Crossing', 'Finishing', 'Heading Accuracy', 'Short Passing', 'Volleys', 'Dribbling', 'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'Standing Tackle', 'Sliding Tackle', 'GK Diving', 'GK Handling', 'GK Kicking', 'GK Positioning', 'GK Reflexes']

In [64]:
def addwrong(a):
    b=a.replace('+',' ').replace('-',' ').split()
    count=[]
    for i in range(len(a)):
        if a[i]=='+':
            count+=[1]
        elif a[i]=='-':
            count+=[-1]
    w=[b[0]]
    n=[]
    f=0
    c=0
    for i in range(1,len(b)):
        if f==0:
            if b[i].isdigit():
                n+=[int(b[i])]
                f=1
            else:
                w[-1]=w[-1]+' '+b[i]
        else:
            if b[i].isdigit():
                n[-1]+=int(b[i])*count[c]
                c+=1
            else:
                w+=[b[i]]
                f=0
    return w,n
def Dkwrong(a):
    b=re.sub("\D", ' ', a).split()
    return [int(i) for i in b]

In [65]:
def cleardict(v,k):
    v2={}
    for i in k:
        v2[i]='Non'
    for i in v.keys():
        if '+' in i:
            w,n=addwrong(i)
            v2[w[0]]=v[i]
            for i in range(len(n)):
                v2[w[i+1]]=n[i]
        elif 'GK Handling'in i:
            n=Dkwrong(i)
            v2['GK Handling']=v[i]
            v2['GK Kicking']=n[0]
            v2['GK Positioning']=n[1]
        else:
            v2[i]=v[i]
    return v2

In [66]:
def main2Series(main,k):
    main2={}
    for i in main.keys():
        v2= cleardict(main[i],k)
        main2[i]=v2
    names=[i for i in main2.keys()]
    dic={}
    dic['name']=names
    for i in k:
        dic[i]=[]
    for i in names:
        for j in k:
            dic[j]+=[main2[i][j]]
    return pd.DataFrame(dic)

In [67]:
s= main2Series(main,k)

In [68]:
s

,name,Crossing,Finishing,Heading Accuracy,Short Passing,Volleys,Dribbling,Curve,FK Accuracy,Long Passing,...,Penalties,Composure,Marking,Standing Tackle,Sliding Tackle,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes
0,L. Messi,86,95,70,92,86,97,93,94,89,...,75,96,33,28,26,6,11,15,14,8
1,Cristiano Ronaldo,84,94,89,81,87,88,81,76,77,...,85,95,28,31,23,7,11,15,14,11
2,Neymar Jr,83,87,62,84,84,96,88,87,80,...,81,94,27,24,33,9,9,15,15,11
3,De Gea,17,13,21,50,13,18,21,19,51,...,40,71,25,21,13,90,85,85,89,94
4,K. De Bruyne,93,82,55,92,82,86,85,83,91,...,79,90,68,58,51,15,13,5,10,13
5,E. Hazard,81,84,61,89,80,95,83,79,83,...,86,91,34,27,22,11,12,6,8,8
6,L. Modrić,Non,Non,Non,Non,Non,90,85,78,88,...,82,90,68,76,73,13,9,7,14,9
7,L. Suárez,77,92,82,82,90,86,86,84,69,...,85,85,62,45,38,27,25,31,33,37
8,H. Kane,75,94,86,82,85,80,78,68,83,...,90,91,56,36,38,8,10,11,14,11
9,J. Oblak,13,11,15,29,13,12,13,14,26,...,11,70,27,12,18,86,92,78,88,89


In [ ]:

temp=url_main+name['L. Modrić']
page = requests.get(temp)
soup = bs(page.content, 'lxml')
th=soup.find_all('ul',{'class': 'pl'})
l=[]
for j,i in enumerate(th):
    if j!=0 and j!=1 and j!=2 and j!=10:
        l.append(i.text.strip().split())
l

In [ ]:
d={}
i=0
t=0
for u in range(len(l)):
    i=0
    t=0
    while(i<len(l[u])):
        if l[u][i].isdigit():
            j=i+1
            if j==len(l[u])-1:
                d[' '.join(l[u][i+1:min(len(l[u]),j+1)])]=int(l[u][i])
                print(l[u][i])
                break
            else:
                while(l[u][j].isdigit()==False):
                        j=j+1
                        if j==len(l[u])-1:
                            d[' '.join(l[u][i+1:min(len(l[u]),j+1)])]=int(l[u][i])
                            t=-1
                            break
                if t==-1:
                    break
                d[' '.join(l[u][i+1:min(len(l[u]),j)])]=int(l[u][i])
                i=j
                print(l[u][i])
        else:
            i=i+1

In [ ]:
d1={}
d2={}
d1['sharath']=1

In [ ]:
d2['yes']=d1

In [ ]:
d2